<a href="https://colab.research.google.com/github/imabari/ehime-covid-bot/blob/main/ehime_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# インストール

In [ ]:
!pip install pdfminer.six

# プログラム

## 設定

In [ ]:
import datetime
import pathlib
import re
import unicodedata
from urllib.parse import urljoin

In [ ]:
from pdfminer.high_level import extract_text
import requests
import tweepy
from bs4 import BeautifulSoup

In [ ]:
# 愛媛県の新型コロナウイルス感染症に関する情報
url = "https://www.pref.ehime.jp/h25500/kansen/covid19.html"

## 関数

In [ ]:
# スクレイピング
def fetch_soup(url, parser="html.parser"):

    r = requests.get(url)
    r.raise_for_status()

    soup = BeautifulSoup(r.content, parser)

    return soup

In [ ]:
# PDFファイルからテキスト抽出
def fetch_text(url):

    r = requests.get(url)

    p = pathlib.Path(pathlib.PurePath(url).name)

    with p.open(mode="wb") as fw:
        fw.write(r.content)

    text = extract_text(p)

    return unicodedata.normalize("NFKC", text)

## スクレイピング

In [ ]:
soup = fetch_soup(url)

In [ ]:
# 記事のリンクを抽出
tags = soup.select("div#tmp_contents > ul > li > a")

## 新型コロナウイルスの感染の確認等について

In [ ]:
# 最新記事は上から4番目
tag1 = tags[3]

In [ ]:
# タイトル確認
title1 = tag1.get_text(strip=True)
print(title1)

In [ ]:
# PDFのURLを作成
link1 = urljoin(url, tag1.get("href"))

In [ ]:
# PDFファイルをダウンロード
text1 = fetch_text(link1)

In [ ]:
# テキスト確認
print(text1)

# 出力

### 全体

In [ ]:
s1 = ""
m1 = re.search("昨日\((.+)\)、県内で新型コロナウイルスの陽性者が(.+)名確認されました。", text1)

if m1:
    s1 = m1.group(0).replace(" ", "")

print(s1)

### 既存事例

In [ ]:
s2 = ""
m2 = re.search("濃厚接触者及び接触者の検査により判明した陽性者数 *:(.+)名", text1)

if m2:
    s2 = m2.group(0).replace(" ", "")

print(s2)

### 新規事例

In [ ]:
s3 = ""
m3 = re.search("上記以外の検査により新たに判明した陽性者数 *:(.+)名", text1)

if m3:
    s3 = m3.group(0).replace(" ", "")

print(s3)

### 文章結合

In [ ]:
print(f"{s1}\n{s2}\n{s3}")

# 愛媛県内の状況

In [ ]:
# 記事は上から3番目
tag2 = tags[2]

In [ ]:
# タイトル確認
title2 = tag2.get_text(strip=True)
print(title2)

In [ ]:
# PDFのURLを作成
link2 = urljoin(url, tag2.get("href"))

In [ ]:
# PDFファイルをダウンロード
text2 = fetch_text(link2)

In [ ]:
# テキスト確認
print(text2)

In [ ]:
s4 = ""

m4 = re.search("^新規事例\n(\d+)事例合計", text2, re.MULTILINE | re.DOTALL)

if m4:
    s4 = m4.group(0).replace(" ", "")
print(s4)

# 抽出

In [ ]:
result = {}

In [ ]:
result["日付"] = m1.group(1).replace(" ", "")
result["全体人数"] = m1.group(2).replace(" ", "") if m1 else 0
result["既存人数"] = m2.group(1).replace(" ", "") if m2 else 0
result["新規人数"] = m3.group(1).replace(" ", "") if m3 else 0
result["新規事例"] = m4.group(1).replace(" ", "") if m4 else 0

In [ ]:
print(result)

In [ ]:
twit = f'{result["日付"]}\n全体{result["全体人数"]}名\n既存事例{result["既存人数"]}名\n新規{result["新規事例"]}事例{result["新規人数"]}名'
print(twit)